In [1]:
import pandas as pd
import numpy as np
import unidecode

### Scrapeo a wikipedia (HTML)

In [3]:
import pandas as pd
import requests

url = "https://es.wikipedia.org/wiki/Anexo:XLVIII_Legislatura_de_la_Cámara_de_Representantes_de_Uruguay_(2015-2020)"

# Hacer request simulando un navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza un error si no se puede acceder

# Leer tablas con pandas desde el contenido HTML
tablas = pd.read_html(response.text)

# Mostrar la primera tabla
df = tablas[0]

C:\Users\PC\AppData\Local\Temp\ipykernel_5184\3232857856.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablas = pd.read_html(response.text)


In [4]:
leg_48 = tablas[2]

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://biblioteca.parlamento.gub.uy:8008/biografias/busqueda?legislatura=47"
headers = {"User-Agent": "Mozilla/5.0"}
html = requests.get(url, headers=headers).text

print(html[:2000])  # inspeccioná si ya aparece el nombre "ABDALA SCHWARZ" etc.

### Scrapeo a la página de la bliblioteca del parlamento

In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

opts = Options()
opts.add_argument("--headless=new")
driver = webdriver.Chrome(options=opts)

driver.get("https://biblioteca.parlamento.gub.uy:8008//biografias/busqueda?legislatura=47&page=40")

wait = WebDriverWait(driver, 10)
cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card")))

rows = []
for c in cards:
    # Nombre
    try:
        nombre = c.find_element(By.CSS_SELECTOR, "h4.card-title a").text.strip()
    except:
        nombre = ""

    # Fecha nacimiento
    try:
        fecha = c.find_element(By.XPATH, ".//*[contains(text(), 'Fecha de nacimiento')]").text.strip()
    except:
        fecha = ""

    # Cargos: recorrer filas de la tabla
    cargos = []
    try:
        filas = c.find_elements(By.CSS_SELECTOR, ".card-datos-legislativos table tr")
        for fila in filas:
            cargos.append(fila.text.strip())
    except:
        pass

    rows.append([nombre, fecha, " | ".join(cargos)])

driver.quit()

# Guardar CSV
with open("legisladores_leg47_40.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Nombre", "Fecha nacimiento", "Cargos legislativos"])
    writer.writerows(rows)

print("Guardado:", len(rows), "legisladores")


Guardado: 8 legisladores


In [89]:
import glob

# Buscar todos los CSV en la carpeta actual que empiecen con legisladores_leg47_
files = glob.glob("legisladores_leg47_*.csv")

# Leer y concatenar todos
df_list = [pd.read_csv(f) for f in files]
df = pd.concat(df_list, ignore_index=True)

print("Total filas:", len(df))
print(df.head())

Total filas: 398
                                   Nombre                 Fecha nacimiento  \
0            ABDALA SCHWARZ, PABLO DANIEL  Fecha de nacimiento: 14/07/1966   
1                   ABREU BONILLA, SERGIO  Fecha de nacimiento: 12/11/1945   
2                    ABT MURIONDO, ANDRÉS  Fecha de nacimiento: 11/08/1973   
3  ACORDAGOITÍA AMARELLA, OSVALDO ROSENDO  Fecha de nacimiento: 21/04/1932   
4            ACOSTA MARTIRENA, AURO RAMÓN  Fecha de nacimiento: 04/05/1964   

                                 Cargos legislativos  
0   |  |  |  |  |  | 2025 - en actividad DIPUTADO...  
1   |  |  |  |  | 2020 - 2020 SENADOR (PARTIDO NA...  
2   |  |  | 2020 - 2021 DIPUTADO (PARTIDO NACIONA...  
3   | 2010 - 2010 DIPUTADO (FRENTE AMPLIO - Suple...  
4   |  | 2015 - 2019 DIPUTADO (PARTIDO NACIONAL -...  


In [88]:
df_original = df.copy()
df["Cargos legislativos"] = df["Cargos legislativos"].str.split("|")
df = df.explode("Cargos legislativos")

# limpiar espacios
df["Cargos legislativos"] = df["Cargos legislativos"].str.strip()

# ahora podés parsear cada cargo en columnas
# ejemplo: separar "2025 - en actividad DIPUTADO (PARTIDO NACIONAL - Titular)"
df[["Periodo", "Resto"]] = df["Cargos legislativos"].str.split(" ", n=1, expand=True)

# intentar extraer partido y si fue titular/suplente
df["Cargo"] = df["Resto"].str.extract(r"(DIPUTADO|SENADOR|VICEPRESIDENTE.*)")
df["Partido"] = df["Resto"].str.extract(r"\((.*?) -")
df["Condición"] = df["Resto"].str.extract(r"-\s*(Titular|Suplente)\)")

# opcional: limpiar columnas
df = df[["Nombre", "Fecha nacimiento", "Periodo", "Cargo", "Partido", "Condición"]]

In [120]:

# 1. Separar cargos en filas
df["Cargos legislativos"] = df["Cargos legislativos"].str.split("|")
df = df.explode("Cargos legislativos")

# 2. Limpiar espacios
df["Cargos legislativos"] = df["Cargos legislativos"].str.strip()

# 3. Extraer el periodo completo (ej: "2010 - 2015" o "2025 - en actividad")
df["Periodo"] = df["Cargos legislativos"].str.extract(r"^([0-9]{4}\s*-\s*(?:[0-9]{4}|en actividad))")

# 4. Extraer el cargo (DIPUTADO, SENADOR, VICEPRESIDENTE, etc.)
df["Cargo"] = df["Cargos legislativos"].str.extract(r"(DIPUTADO|DIPUTADA|SENADOR|SENADORA|VICEPRESIDENTE.*)")

# 5. Extraer el partido
df["Partido"] = df["Cargos legislativos"].str.extract(r"\((.*?) -")

# 6. Extraer condición (Titular o Suplente)
df["Condición"] = df["Cargos legislativos"].str.extract(r"-\s*(Titular|Suplente)\)")

# 7. Seleccionar columnas limpias
df_limpio = df[["Nombre", "Fecha nacimiento", "Periodo", "Cargo", "Partido", "Condición"]]

In [121]:
df_limpio = df_limpio[df_limpio["Periodo"].notna()]

In [ ]:
import unidecode
#df_limpio = df_limpio.applymap(unidecode.unidecode) da error

In [122]:
df_limpio["Nombre"] = df_limpio["Nombre"].apply(unidecode.unidecode)
df_limpio["Fecha nacimiento"] = df_limpio["Fecha nacimiento"].str.replace("Fecha de nacimiento:", "")

In [123]:
df_limpio[["Inicio", "Fin"]] = df_limpio["Periodo"].str.split("-", n=1, expand=True)

# limpiar espacios
df_limpio["Inicio"] = df_limpio["Inicio"].str.strip()
df_limpio["Fin"] = df_limpio["Fin"].str.strip()

# convertir a número (en actividad -> 9999)
df_limpio["Inicio"] = pd.to_numeric(df_limpio["Inicio"], errors="coerce")
df_limpio["Fin"] = df_limpio["Fin"].replace("en actividad", "9999")
df_limpio["Fin"] = pd.to_numeric(df_limpio["Fin"], errors="coerce")

In [125]:
mask = (df_limpio["Inicio"] >= 2010) & (df_limpio["Fin"] <= 2015)
df_periodo = df_limpio[mask]

In [129]:
df_periodo = df_periodo[df_periodo["Cargo"].notna()]

In [145]:
df_periodo.to_csv("C:/Users/PC/Desktop/pasantia_CP/pasantia_UMAD/leg47_biblioteca.csv", encoding="utf-8")

In [ ]:
df_periodo[["Apellidos", "Nombres"]] = df_periodo["Nombre"].str.split(",", n=1, expand=True)

# limpiar espacios
df_periodo["Apellidos"] = df_periodo["Apellidos"].str.strip()
df_periodo["Nombres"] = df_periodo["Nombres"].str.strip()

# 2. Separar apellidos
ap_split = df_periodo["Apellidos"].str.split(" ", n=1, expand=True)
df_periodo["primer_apellido"] = ap_split[0]
df_periodo["segundo_apellido"] = ap_split[1]

# 3. Separar nombres
nom_split = df_periodo["Nombres"].str.split(" ", n=1, expand=True)
df_periodo["primer_nombre"] = nom_split[0]
df_periodo["segundo_nombre"] = nom_split[1]


In [140]:
import datetime

In [144]:
df_periodo["Fecha nacimiento"] = df_periodo["Fecha nacimiento"].str.replace("/", "-")

C:\Users\PC\AppData\Local\Temp\ipykernel_5184\2849931409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_periodo["Fecha nacimiento"] = df_periodo["Fecha nacimiento"].str.replace("/", "-")
